# DIY Disease Tracking Dashboard


The dashboard presents daily reports of Covid-19 cases. It presents data for some of the largest cities in England, specifically: Manchester, Birmingham, Leeds, Liverpool, Sheffield, Leicester and Nottingham. You have the option of selecting each city one by one, multiple cities or summarised data of cases reported per day for all of the mentioned above. Please note that the initial graph that is displayed is data from a file, to access the most recent data, click on the "refresh" button and this will download the latest data by accessing an API.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json
from APIWrapper import APIwrapper
import matplotlib.dates as mdates

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
jsondata = {}
#loads saved file to generate initial graph
with open("daily_cases_cities.json", "rt") as INFILE:
    jsondata = json.load(INFILE)

In [4]:
df = None # data will be stored in here
def wrangle_data(rawdata): 
    global df
    """Process raw API or JSON data into a structured DataFrame."""
    # Temporary dictionary to organize data by date and city
    data = {}
    for city, dataset in rawdata.items():
        for entry in dataset:
            date = entry['date']
            value = entry['metric_value']

            if date not in data:
                data[date] = {}
            data[date][city] = value

    dates = sorted(data.keys())
    #creates data frame sorting by date
    df = pd.DataFrame(index=pd.to_datetime(dates))
    # Populate the DataFrame with the case data for each city
    for date, city_data in data.items():
        for city, value in city_data.items():
            df.loc[pd.to_datetime(date), city] = value

    df.fillna(0.0, inplace=True)
    df['Total Cases'] = df.sum(axis=1)
    return df

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """Fetch data from UKHSA API and return structured data."""
    geography = ["Manchester", "Birmingham", "Leeds", "Liverpool", "Sheffield", "Leicester", "Nottingham"]
    structure = {
        "theme": "infectious_disease",
        "sub_theme": "respiratory",
        "topic": "COVID-19",
        "geography_type": "Upper%20Tier%20Local%20Authority",
        "metric": "COVID-19_cases_casesByDay",
    }
    all_city_data = {}
    #iteration over each city data, fetches it and stores returning as aggregated data
    try:
        for city in geography:
            structure["geography"] = city
            try:
                api = APIwrapper(**structure)
                city_data = api.get_all_pages({})
                if city_data:
                    all_city_data[city] = [
                        {"date": entry["date"], "metric_value": entry["metric_value"]}
                        for entry in city_data
                    ]
            except Exception as e:
                continue
        return all_city_data
    except Exception as e:
        return {}

In [6]:
def plot_cases(selected_cities):    
    """Plot daily COVID-19 cases for the selected cities.
    This function generates a line plot for the specified cities, using data from dataframe."""
    
    global df
    # this ensure is total cases exists, otherwise cases are summed and this exist, due to issues with refresh this was added in
    if 'Total Cases' not in df.columns:
        df['Total Cases'] = df.sum(axis=1)
    
    columns_to_plot = [col for col in selected_cities if col in df.columns]
    if not columns_to_plot:
        columns_to_plot = ["Total Cases"]

    plt.figure(figsize=(20, 8))
    ax = df[columns_to_plot].plot(ax=plt.gca(), lw=2)
    ax.set_title("Daily COVID-19 Cases", fontsize=16)
    ax.set_xlabel("Date", fontsize=14)
    ax.set_ylabel("Daily Cases", fontsize=14)
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    plt.xticks(rotation=45)
    ax.legend(title="Cities", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()


def refresh_graph(selected_cities=None):
    """ refreshes the graph, set total cases as default view until user changes it"""
    if not selected_cities:
        selected_cities = ["Total Cases"]
    plot_cases(selected_cities)

last_fetched_data = None
#
def api_button_callback(button):
    """Callback function for the API refresh button.
    This function is triggered when the "Refresh" button is clicked. It fetches new data from the API,
    updates the global JSON data and DataFrame, refreshes the widget options, and updates the graph."""
    global jsondata, df
    try:
        apidata = access_api()
        if apidata:
            jsondata.update(apidata)
            df = wrangle_data(jsondata)
            city_widget.options = ["Total Cases"] + [col for col in df.columns if col != "Total Cases"]
            refresh_graph(city_widget.value)
            apibutton.icon = "check"
            apibutton.description = "Data Refreshed"
        else:
            raise ValueError("No data returned from API.")
    except Exception as e:
        apibutton.icon = "times"
        apibutton.description = "Error Fetching Data"

# refresh button widget
apibutton = wdg.Button(
    description="Refresh",
    disabled=False,
    button_style="success",
    tooltip="Fetch the latest data",
    icon="download",
)
apibutton.on_click(api_button_callback)
# widget for selection of cities and total cases between them
city_widget = wdg.SelectMultiple(
    options=["Total Cases"] + [col for col in wrangle_data(jsondata).columns if col != "Total Cases"],
    value=["Total Cases"],
    description="Cities",
    disabled=False,
)

graph_output = wdg.interactive_output(
    refresh_graph,
    {"selected_cities": city_widget},
)

# Display Widgets
display(apibutton, city_widget, graph_output)


Button(button_style='success', description='Refresh', icon='download', style=ButtonStyle(), tooltip='Fetch the…

SelectMultiple(description='Cities', index=(0,), options=('Total Cases', 'Manchester', 'Birmingham', 'Leeds', …

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."